In [476]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from tqdm.auto import tqdm
from sklearn.metrics import f1_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/submission/submission_format.csv
/kaggle/input/x-validation/test_set_features.csv
/kaggle/input/y-train/training_set_labels.csv
/kaggle/input/x-train/training_set_features.csv
/kaggle/input/week4-practice-hackathon-2024/sample_submission.csv
/kaggle/input/week4-practice-hackathon-2024/Train.csv
/kaggle/input/week4-practice-hackathon-2024/Test.csv


**Here's an outline of the project:**
  
    1.Download the dataset
    2.Some exploratory data analysis
    3.Some pre-processing
    4.Training a baseline model
    5.According to the score doing some feature engineering
    6.Train different models
    7.Fine tune the hyper parameters of the best models
    8.Submission file

# 1.Download the Dataset

 

***Loading Training Set***

In [477]:
df1 = pd.read_csv("/kaggle/input/x-train/training_set_features.csv")
df1

,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,26702,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,qufhixun,Non-MSA,0.0,0.0,NaN,NaN
26703,26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Rent,Employed,lzgpxyit,"MSA, Principle City",1.0,0.0,fcxhlnwr,cmhcxjea
26704,26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,NaN,Not Married,Own,NaN,lzgpxyit,"MSA, Not Principle City",0.0,0.0,NaN,NaN
26705,26705,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,"<= $75,000, Above Poverty",Married,Rent,Employed,lrircsnp,Non-MSA,1.0,0.0,fcxhlnwr,haliazsg


In [478]:
df2=pd.read_csv("/kaggle/input/y-train/training_set_labels.csv")
df2

,respondent_id,xyz_vaccine,seasonal_vaccine
0,0,0,0
1,1,0,1
2,2,0,0
3,3,0,1
4,4,0,0
...,...,...,...
26702,26702,0,0
26703,26703,0,0
26704,26704,0,1
26705,26705,0,0


In [479]:
df = pd.concat([df1, df2], axis=1)
df

,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,respondent_id,xyz_vaccine,seasonal_vaccine
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN,0,0,0
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe,1,0,1
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo,2,0,0
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN,3,0,1
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,26702,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,Not in Labor Force,qufhixun,Non-MSA,0.0,0.0,NaN,NaN,26702,0,0
26703,26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,Employed,lzgpxyit,"MSA, Principle City",1.0,0.0,fcxhlnwr,cmhcxjea,26703,0,0
26704,26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,NaN,lzgpxyit,"MSA, Not Principle City",0.0,0.0,NaN,NaN,26704,0,1
26705,26705,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,Employed,lrircsnp,Non-MSA,1.0,0.0,fcxhlnwr,haliazsg,26705,0,0


***Loading Test Set***

In [480]:
test_df=pd.read_csv("/kaggle/input/x-validation/test_set_features.csv")
test_df

,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,26707,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"> $75,000",Not Married,Rent,Employed,mlyzmhmf,"MSA, Not Principle City",1.0,0.0,atmlpfrs,hfxkjkmi
1,26708,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,Non-MSA,3.0,0.0,atmlpfrs,xqwwgdyp
2,26709,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,"> $75,000",Married,Own,Employed,lrircsnp,Non-MSA,1.0,0.0,nduyfdeo,pvmttkik
3,26710,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,lrircsnp,"MSA, Not Principle City",1.0,0.0,NaN,NaN
4,26711,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,lzgpxyit,Non-MSA,0.0,1.0,fcxhlnwr,mxkfnird
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26703,53410,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,dqpwygqj,"MSA, Principle City",1.0,1.0,NaN,NaN
26704,53411,3.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,Below Poverty,Married,Rent,Employed,qufhixun,Non-MSA,1.0,3.0,fcxhlnwr,vlluhbov
26705,53412,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,qufhixun,"MSA, Not Principle City",1.0,0.0,NaN,NaN
26706,53413,3.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,bhuqouqj,"MSA, Not Principle City",1.0,0.0,NaN,NaN


# 2.Exploratory Data Analysis:

***Training Set***

In [481]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26707 entries, 0 to 26706
Data columns (total 39 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   respondent_id                26707 non-null  int64  
 1   xyz_concern                  26615 non-null  float64
 2   xyz_knowledge                26591 non-null  float64
 3   behavioral_antiviral_meds    26636 non-null  float64
 4   behavioral_avoidance         26499 non-null  float64
 5   behavioral_face_mask         26688 non-null  float64
 6   behavioral_wash_hands        26665 non-null  float64
 7   behavioral_large_gatherings  26620 non-null  float64
 8   behavioral_outside_home      26625 non-null  float64
 9   behavioral_touch_face        26579 non-null  float64
 10  doctor_recc_xyz              24547 non-null  float64
 11  doctor_recc_seasonal         24547 non-null  float64
 12  chronic_med_condition        25736 non-null  float64
 13  child_under_6_mo

In [482]:
df.describe()

,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,opinion_xyz_risk,opinion_xyz_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,household_adults,household_children,respondent_id,xyz_vaccine,seasonal_vaccine
count,26707.000000,26615.000000,26591.000000,26636.000000,26499.000000,26688.000000,26665.000000,26620.00000,26625.000000,26579.000000,...,26319.000000,26312.000000,26245.000000,26193.000000,26170.000000,26458.000000,26458.000000,26707.000000,26707.000000,26707.000000
mean,13353.000000,1.618486,1.262532,0.048844,0.725612,0.068982,0.825614,0.35864,0.337315,0.677264,...,2.342566,2.357670,4.025986,2.719162,2.118112,0.886499,0.534583,13353.000000,0.212454,0.465608
std,7709.791156,0.910311,0.618149,0.215545,0.446214,0.253429,0.379448,0.47961,0.472802,0.467531,...,1.285539,1.362766,1.086565,1.385055,1.332950,0.753422,0.928173,7709.791156,0.409052,0.498825
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6676.500000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,...,1.000000,1.000000,4.000000,2.000000,1.000000,0.000000,0.000000,6676.500000,0.000000,0.000000
50%,13353.000000,2.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.00000,0.000000,1.000000,...,2.000000,2.000000,4.000000,2.000000,2.000000,1.000000,0.000000,13353.000000,0.000000,0.000000
75%,20029.500000,2.000000,2.000000,0.000000,1.000000,0.000000,1.000000,1.00000,1.000000,1.000000,...,4.000000,4.000000,5.000000,4.000000,4.000000,1.000000,1.000000,20029.500000,0.000000,1.000000
max,26706.000000,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,3.000000,3.000000,26706.000000,1.000000,1.000000


***Test Set***

In [483]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26708 entries, 0 to 26707
Data columns (total 36 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   respondent_id                26708 non-null  int64  
 1   xyz_concern                  26623 non-null  float64
 2   xyz_knowledge                26586 non-null  float64
 3   behavioral_antiviral_meds    26629 non-null  float64
 4   behavioral_avoidance         26495 non-null  float64
 5   behavioral_face_mask         26689 non-null  float64
 6   behavioral_wash_hands        26668 non-null  float64
 7   behavioral_large_gatherings  26636 non-null  float64
 8   behavioral_outside_home      26626 non-null  float64
 9   behavioral_touch_face        26580 non-null  float64
 10  doctor_recc_xyz              24548 non-null  float64
 11  doctor_recc_seasonal         24548 non-null  float64
 12  chronic_med_condition        25776 non-null  float64
 13  child_under_6_mo

In [484]:
test_df.describe()

,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,health_worker,health_insurance,opinion_xyz_vacc_effective,opinion_xyz_risk,opinion_xyz_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,household_adults,household_children
count,26708.000000,26623.000000,26586.000000,26629.000000,26495.000000,26689.000000,26668.000000,26636.000000,26626.000000,26580.000000,...,25919.000000,14480.000000,26310.000000,26328.000000,26333.000000,26256.000000,26209.000000,26187.000000,26483.000000,26483.000000
mean,40060.500000,1.623145,1.266042,0.049645,0.729798,0.069279,0.826084,0.351517,0.337227,0.683747,...,0.111501,0.887914,3.844622,2.326838,2.360612,4.024832,2.708688,2.143392,0.894310,0.543745
std,7710.079831,0.902755,0.615617,0.217215,0.444072,0.253934,0.379045,0.477453,0.472772,0.465022,...,0.314758,0.315483,1.007570,1.275636,1.359413,1.083204,1.376045,1.339102,0.754244,0.935057
min,26707.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,33383.750000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,3.000000,1.000000,1.000000,4.000000,2.000000,1.000000,0.000000,0.000000
50%,40060.500000,2.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,...,0.000000,1.000000,4.000000,2.000000,2.000000,4.000000,2.000000,2.000000,1.000000,0.000000
75%,46737.250000,2.000000,2.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,1.000000,5.000000,4.000000,4.000000,5.000000,4.000000,4.000000,1.000000,1.000000
max,53414.000000,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,3.000000,3.000000


**Observations:**
   * there are Nan values,we can remove or impute them
   

# 3.Prepare Dataset for Training:

   *  Split Training and Validation Set
   *  Fill/Remove missing values or discrepancies
   *  Extract Inputs & Outputs
                
            

# Split Training & Validation Set

 We'll set aside 20% of the training data as the validation set,to evaluate the models we train on previously unseen data.
 
 Since there is no time factor,we can pick a random 20% fraction

In [485]:
from sklearn.model_selection import train_test_split

In [486]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [487]:
len(train_df), len(val_df)

(21365, 5342)

# Fill/Remove the Missing Values
  
  There are no missing values in our sample , but if there were,we could drop that rows or in the time of pre-processing we can also impute some values there.

In [488]:
train_df = train_df.dropna()
val_df = val_df.dropna()

In [489]:
train_df=train_df.drop(columns=['respondent_id','hhs_geo_region','employment_industry','employment_occupation','employment_status'])
val_df=val_df.drop(columns=['respondent_id','hhs_geo_region','employment_industry','employment_occupation','employment_status'])

# Extract Inputs and Outputs

In [490]:
train_df.columns

Index(['xyz_concern', 'xyz_knowledge', 'behavioral_antiviral_meds',
       'behavioral_avoidance', 'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_xyz', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'health_insurance', 'opinion_xyz_vacc_effective', 'opinion_xyz_risk',
       'opinion_xyz_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group',
       'education', 'race', 'sex', 'income_poverty', 'marital_status',
       'rent_or_own', 'census_msa', 'household_adults', 'household_children',
       'xyz_vaccine', 'seasonal_vaccine'],
      dtype='object')

In [491]:
input_cols = train_df.columns[0:-2]

In [492]:
target_cols1 =['xyz_vaccine']

In [493]:
target_cols2 = ['seasonal_vaccine']

# Training

In [494]:
train_inputs = train_df[input_cols]

In [495]:
train_targets1 = train_df[target_cols1]

In [496]:
train_targets2 = train_df[target_cols2]

In [497]:
train_inputs 

,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_xyz,...,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,census_msa,household_adults,household_children
5449,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,35 - 44 Years,College Graduate,White,Male,"> $75,000",Not Married,Own,"MSA, Not Principle City",1.0,3.0
14335,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,45 - 54 Years,College Graduate,Black,Female,"> $75,000",Not Married,Own,"MSA, Principle City",2.0,2.0
17854,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,35 - 44 Years,College Graduate,White,Female,"<= $75,000, Above Poverty",Not Married,Own,"MSA, Not Principle City",0.0,1.0
13073,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,35 - 44 Years,Some College,White,Male,"> $75,000",Married,Own,"MSA, Principle City",1.0,0.0
4275,2.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,55 - 64 Years,12 Years,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Non-MSA,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1685,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,18 - 34 Years,College Graduate,White,Female,"> $75,000",Not Married,Own,"MSA, Principle City",3.0,0.0
22118,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,35 - 44 Years,College Graduate,White,Female,"> $75,000",Married,Rent,"MSA, Principle City",0.0,0.0
11284,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,45 - 54 Years,Some College,White,Female,"> $75,000",Married,Own,"MSA, Not Principle City",1.0,0.0
15795,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,35 - 44 Years,College Graduate,Black,Female,"> $75,000",Married,Own,"MSA, Principle City",1.0,0.0


In [498]:
train_targets1

,xyz_vaccine
5449,0
14335,0
17854,1
13073,0
4275,0
...,...
1685,1
22118,0
11284,0
15795,0


In [499]:
train_targets2

,seasonal_vaccine
5449,0
14335,0
17854,1
13073,0
4275,0
...,...
1685,1
22118,0
11284,0
15795,0


# Validation

In [500]:
val_inputs = val_df[input_cols]

In [501]:
val_targets1 = val_df[target_cols1]

In [502]:
val_targets2 = val_df[target_cols2]

In [503]:
val_inputs 

,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_xyz,...,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,census_msa,household_adults,household_children
23353,2.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,35 - 44 Years,College Graduate,White,Female,"> $75,000",Married,Own,Non-MSA,1.0,2.0
19331,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,45 - 54 Years,College Graduate,White,Female,"> $75,000",Married,Own,"MSA, Not Principle City",2.0,0.0
6027,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,18 - 34 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Not Married,Own,"MSA, Principle City",1.0,0.0
15714,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,35 - 44 Years,College Graduate,White,Male,"> $75,000",Married,Own,"MSA, Not Principle City",2.0,1.0
7757,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,45 - 54 Years,Some College,White,Male,"> $75,000",Married,Rent,"MSA, Not Principle City",1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9242,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,...,35 - 44 Years,Some College,White,Female,Below Poverty,Not Married,Rent,Non-MSA,2.0,2.0
26500,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,35 - 44 Years,Some College,White,Female,"> $75,000",Married,Own,Non-MSA,2.0,1.0
25132,1.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,...,35 - 44 Years,College Graduate,White,Female,"> $75,000",Married,Own,"MSA, Not Principle City",1.0,2.0
20086,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,45 - 54 Years,Some College,White,Male,"<= $75,000, Above Poverty",Not Married,Own,"MSA, Not Principle City",0.0,0.0


In [504]:
val_targets1

,xyz_vaccine
23353,0
19331,0
6027,0
15714,0
7757,0
...,...
9242,0
26500,0
25132,1
20086,0


In [505]:
val_targets2

,seasonal_vaccine
23353,0
19331,1
6027,1
15714,0
7757,0
...,...
9242,0
26500,1
25132,1
20086,0


# Test

In [506]:
test_inputs = test_df[input_cols]

In [507]:
test_inputs

,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_xyz,...,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,census_msa,household_adults,household_children
0,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,35 - 44 Years,College Graduate,Hispanic,Female,"> $75,000",Not Married,Rent,"MSA, Not Principle City",1.0,0.0
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,18 - 34 Years,12 Years,White,Male,Below Poverty,Not Married,Rent,Non-MSA,3.0,0.0
2,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,...,55 - 64 Years,College Graduate,White,Male,"> $75,000",Married,Own,Non-MSA,1.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,65+ Years,12 Years,White,Female,"<= $75,000, Above Poverty",Married,Own,"MSA, Not Principle City",1.0,0.0
4,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,35 - 44 Years,12 Years,Black,Female,"<= $75,000, Above Poverty",Not Married,Own,Non-MSA,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26703,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,35 - 44 Years,NaN,White,Female,NaN,NaN,NaN,"MSA, Principle City",1.0,1.0
26704,3.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,18 - 34 Years,12 Years,White,Male,Below Poverty,Married,Rent,Non-MSA,1.0,3.0
26705,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,18 - 34 Years,Some College,White,Female,Below Poverty,Not Married,Rent,"MSA, Not Principle City",1.0,0.0
26706,3.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,55 - 64 Years,Some College,White,Female,"<= $75,000, Above Poverty",Married,Own,"MSA, Not Principle City",1.0,0.0


# Scaling & One-Hot Encoding

 * this data needs some one hot encoding and numerical encoding

In [508]:
train_inputs['income_poverty'] = train_inputs['income_poverty'].replace({
    'Below Poverty':1,
    '<= $75,000, Above Poverty':2,
    '> $75,000':3
})
train_inputs['age_group'] = train_inputs['age_group'].replace({
    '18 - 34 Years':1,
    '35 - 44 Years':2,
    '45 - 54 Years':3,
    '55 - 64 Years':4,
    '65+ Years':5
})
train_inputs['education'] = train_inputs['education'].replace({
    '< 12 Years':1,
    '12 Years':2,
    'Some College':3,
    'College Graduate':4
})

# One-hot encoding
train_inputs = pd.get_dummies(train_inputs, columns=['race'], prefix='race')
train_inputs['sex'] = train_inputs['sex'].replace({
    'Female':0,
    'Male':1    
})
# One-hot encoding
train_inputs = pd.get_dummies(train_inputs, columns=['census_msa'], prefix='census_msa')
train_inputs['marital_status'] = train_inputs['marital_status'].replace({
    'Not Married':0,
    'Married':1    
})
train_inputs['rent_or_own'] = train_inputs['rent_or_own'].replace({
    'Rent':0,
    'Own':1    
})


train_inputs = train_inputs.astype(float)
# import pandas as pd
# from sklearn.impute import SimpleImputer

# # Assuming X_train is your dataset
# # X_train = pd.read_csv('your_dataset.csv') # or however you load your dataset

# # Initialize the SimpleImputer with strategy as 'most_frequent' to use mode
# imputer = SimpleImputer(strategy='most_frequent')

# # Fit the imputer to the dataset and transform the dataset
# train_inputs_imputed = imputer.fit_transform(train_inputs)

# # If you want to convert the numpy array back to a DataFrame
# X_val_imputed = pd.DataFrame(X_val_imputed, columns=X_val.columns)

# # Display the imputed DataFrame
# X_val_imputed
train_inputs

/tmp/ipykernel_33/1686603580.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_inputs['income_poverty'] = train_inputs['income_poverty'].replace({
/tmp/ipykernel_33/1686603580.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_inputs['income_poverty'] = train_inputs['income_poverty'].replace({
/tmp/ipykernel_33/1686603580.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=Fals

,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_xyz,...,rent_or_own,household_adults,household_children,race_Black,race_Hispanic,race_Other or Multiple,race_White,"census_msa_MSA, Not Principle City","census_msa_MSA, Principle City",census_msa_Non-MSA
5449,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,3.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
14335,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
17854,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
13073,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4275,2.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1685,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
22118,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
11284,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
15795,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [509]:
val_inputs['income_poverty'] = val_inputs['income_poverty'].replace({
    'Below Poverty':1,
    '<= $75,000, Above Poverty':2,
    '> $75,000':3
})
val_inputs['age_group'] = val_inputs['age_group'].replace({
    '18 - 34 Years':1,
    '35 - 44 Years':2,
    '45 - 54 Years':3,
    '55 - 64 Years':4,
    '65+ Years':5
})
val_inputs['education'] =val_inputs['education'].replace({
    '< 12 Years':1,
    '12 Years':2,
    'Some College':3,
    'College Graduate':4
})
# One-hot encoding
val_inputs = pd.get_dummies(val_inputs, columns=['race'], prefix='race')
val_inputs['sex'] = val_inputs['sex'].replace({
    'Female':0,
    'Male':1    
})
# One-hot encoding
val_inputs = pd.get_dummies(val_inputs, columns=['census_msa'], prefix='census_msa')
val_inputs['marital_status'] = val_inputs['marital_status'].replace({
    'Not Married':0,
    'Married':1    
})
val_inputs['rent_or_own'] = val_inputs['rent_or_own'].replace({
    'Rent':0,
    'Own':1    
})

val_inputs = val_inputs.astype(float)
# import pandas as pd
# from sklearn.impute import SimpleImputer

# # Assuming X_train is your dataset
# # X_train = pd.read_csv('your_dataset.csv') # or however you load your dataset

# # Initialize the SimpleImputer with strategy as 'most_frequent' to use mode
# imputer = SimpleImputer(strategy='most_frequent')

# # Fit the imputer to the dataset and transform the dataset
# train_inputs_imputed = imputer.fit_transform(train_inputs)

# # If you want to convert the numpy array back to a DataFrame
# X_val_imputed = pd.DataFrame(X_val_imputed, columns=X_val.columns)

# # Display the imputed DataFrame
# X_val_imputed
val_inputs

/tmp/ipykernel_33/1106814435.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  val_inputs['income_poverty'] = val_inputs['income_poverty'].replace({
/tmp/ipykernel_33/1106814435.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_inputs['income_poverty'] = val_inputs['income_poverty'].replace({
/tmp/ipykernel_33/1106814435.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To 

,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_xyz,...,rent_or_own,household_adults,household_children,race_Black,race_Hispanic,race_Other or Multiple,race_White,"census_msa_MSA, Not Principle City","census_msa_MSA, Principle City",census_msa_Non-MSA
23353,2.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
19331,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
6027,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
15714,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,1.0,2.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
7757,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9242,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,2.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
26500,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
25132,1.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,...,1.0,1.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
20086,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


# Train & Evaluate Different Models

We'll train each of the following & submit predictions to kaggle:
     
  * Logistic Regression
  * Random forest
  * Gradient Boosting

In [510]:
from sklearn.metrics import accuracy_score,roc_auc_score

In [511]:
def evaluate1(model):
    train_preds = (model.predict_proba(train_inputs))[:,1].reshape(-1)
    train_ra = roc_auc_score(train_targets1,train_preds)
    val_preds = (model.predict_proba(val_inputs))[:,1].reshape(-1)
    val_ra = roc_auc_score(val_targets1,val_preds)
    return train_ra, val_ra, train_preds, val_preds

In [512]:
def evaluate2(model):
    train_preds = (model.predict_proba(train_inputs))[:,1].reshape(-1)
    train_ra = roc_auc_score(train_targets2,train_preds)
    val_preds = (model.predict_proba(val_inputs))[:,1].reshape(-1)
    val_ra = roc_auc_score(val_targets2,val_preds)
    return train_ra, val_ra, train_preds, val_preds

# Logistic Regression

In [513]:
model_1_lr = LogisticRegression(penalty='elasticnet',solver='saga',random_state=42,n_jobs=-1,l1_ratio=0.30,max_iter=500,class_weight='balanced',C=0.1)

In [514]:
model_2_lr= LogisticRegression(penalty='elasticnet',solver='saga',random_state=42,n_jobs=-1,l1_ratio=0.40,max_iter=300,class_weight=None,C=0.1)

In [515]:
model_1_lr.fit(train_inputs,train_targets1)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=0.1, class_weight='balanced', l1_ratio=0.3, max_iter=500,
                   n_jobs=-1, penalty='elasticnet', random_state=42,
                   solver='saga')

In [516]:
model_2_lr.fit(train_inputs,train_targets2)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=0.1, l1_ratio=0.4, max_iter=300, n_jobs=-1,
                   penalty='elasticnet', random_state=42, solver='saga')

In [517]:
evaluate1(model_1_lr)

(0.8714334418797114,
 0.8603754324278342,
 array([0.03662109, 0.07339106, 0.03358378, ..., 0.39401202, 0.08805747,
        0.26460917]),
 array([0.44077495, 0.46428669, 0.16639879, ..., 0.59969809, 0.35120227,
        0.95125466]))

In [518]:
evaluate2(model_2_lr)

(0.8707947331953263,
 0.8503387472760162,
 array([0.05554192, 0.11622529, 0.05408854, ..., 0.25238343, 0.09137198,
        0.20170867]),
 array([0.32744299, 0.86402012, 0.19303457, ..., 0.77175667, 0.31082407,
        0.62520432]))

# Random Forest

In [519]:
from sklearn.ensemble import RandomForestClassifier

In [520]:
model_1_rf = RandomForestClassifier(random_state=42,n_jobs=-1,n_estimators=1200,max_depth=10,max_leaf_nodes=512,max_features='sqrt',class_weight='balanced',max_samples=0.97,min_impurity_decrease=1e-4)

In [521]:
model_2_rf = RandomForestClassifier(random_state=42,n_jobs=-1,n_estimators=1200,max_depth=12,max_leaf_nodes=512,max_features='sqrt',class_weight=None,max_samples=0.95,min_impurity_decrease=1e-4)

In [522]:
%%time
model_1_rf.fit(train_inputs,train_targets1)

<timed eval>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


CPU times: user 11.1 s, sys: 1.01 s, total: 12.1 s
Wall time: 6.22 s


RandomForestClassifier(class_weight='balanced', max_depth=10,
                       max_leaf_nodes=512, max_samples=0.97,
                       min_impurity_decrease=0.0001, n_estimators=1200,
                       n_jobs=-1, random_state=42)

In [523]:
%%time
model_2_rf.fit(train_inputs,train_targets2)

<timed eval>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


CPU times: user 11.7 s, sys: 937 ms, total: 12.7 s
Wall time: 6.53 s


RandomForestClassifier(max_depth=12, max_leaf_nodes=512, max_samples=0.95,
                       min_impurity_decrease=0.0001, n_estimators=1200,
                       n_jobs=-1, random_state=42)

In [524]:
evaluate1(model_1_rf)

(0.9551392512099023,
 0.8595049055747745,
 array([0.11393801, 0.12043614, 0.17676009, ..., 0.3168234 , 0.13581729,
        0.22649644]),
 array([0.44828391, 0.29231355, 0.1273233 , ..., 0.64022242, 0.16807772,
        0.7986028 ]))

In [525]:
evaluate2(model_2_rf)

(0.9663165185076603,
 0.8565415968361432,
 array([0.096526  , 0.10663984, 0.1886983 , ..., 0.20514584, 0.12961564,
        0.25775438]),
 array([0.27172529, 0.83303823, 0.18652262, ..., 0.8235391 , 0.18498139,
        0.63125716]))

# Gradient Boosting

In [526]:
from xgboost import XGBClassifier

In [527]:
model_1_xgb=XGBClassifier(random_state=42,n_jobs=-1,colsample_bytree=0.5, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=3, n_estimators=1000, reg_alpha=0.5, reg_lambda=10, subsample=0.7, objective="binary:logistic",scale_pos_weight=2.313548387096774)

In [528]:
model_2_xgb=XGBClassifier(random_state=42,n_jobs=-1,colsample_bytree=0.5, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=3, n_estimators=1000, reg_alpha=0.5, reg_lambda=10, subsample=0.8, objective="binary:logistic",scale_pos_weight= 1.1958101752885848)

In [529]:
model_1_xgb.fit(train_inputs,train_targets1)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=-1,
              num_parallel_tree=None, random_state=42, ...)

In [530]:
model_2_xgb.fit(train_inputs,train_targets2)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=-1,
              num_parallel_tree=None, random_state=42, ...)

In [531]:
evaluate1(model_1_xgb)

(0.8919330550707951,
 0.8665598593546192,
 array([0.10333389, 0.09603565, 0.05235376, ..., 0.3838593 , 0.17008859,
        0.22750914], dtype=float32),
 array([0.50291246, 0.37361473, 0.15994105, ..., 0.64871264, 0.24931447,
        0.8977709 ], dtype=float32))

In [532]:
evaluate2(model_2_xgb)

(0.8871006054095398,
 0.8611397290971328,
 array([0.10577905, 0.12303469, 0.05713742, ..., 0.26850483, 0.14300999,
        0.2608265 ], dtype=float32),
 array([0.3205461 , 0.9074826 , 0.20819913, ..., 0.8599132 , 0.2375853 ,
        0.67666924], dtype=float32))

# 3 Model Ensembling

In [533]:
from sklearn.ensemble import VotingClassifier

voting_clf1 = VotingClassifier(estimators=[('model_1_lr', model_1_lr), ('model_1_rf', model_1_rf), ('model_1_xgb', model_1_xgb)], voting='soft')
voting_clf1.fit(train_inputs,train_targets1)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


VotingClassifier(estimators=[('model_1_lr',
                              LogisticRegression(C=0.1, class_weight='balanced',
                                                 l1_ratio=0.3, max_iter=500,
                                                 n_jobs=-1,
                                                 penalty='elasticnet',
                                                 random_state=42,
                                                 solver='saga')),
                             ('model_1_rf',
                              RandomForestClassifier(class_weight='balanced',
                                                     max_depth=10,
                                                     max_leaf_nodes=512,
                                                     max_samples=0.97,
                                                     min_impurity_decrease=0.0001,
                                                     n_estimators=1200,
                                                     n_jobs=-1,
                                                     ra...
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.01, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=3,
                                            max_leaves=None, min_child_weight=3,
                                            missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=1000, n_jobs=-1,
                                            num_parallel_tree=None,
                                            random_state=42, ...))],
                 voting='soft')

In [534]:
from sklearn.ensemble import VotingClassifier

voting_clf2 = VotingClassifier(estimators=[('model_2_lr', model_2_lr), ('model_2_rf', model_2_rf), ('model_2_xgb', model_2_xgb)], voting='soft')
voting_clf2.fit(train_inputs,train_targets2)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


VotingClassifier(estimators=[('model_2_lr',
                              LogisticRegression(C=0.1, l1_ratio=0.4,
                                                 max_iter=300, n_jobs=-1,
                                                 penalty='elasticnet',
                                                 random_state=42,
                                                 solver='saga')),
                             ('model_2_rf',
                              RandomForestClassifier(max_depth=12,
                                                     max_leaf_nodes=512,
                                                     max_samples=0.95,
                                                     min_impurity_decrease=0.0001,
                                                     n_estimators=1200,
                                                     n_jobs=-1,
                                                     random_state=42)),
                             ('model_2_xgb',
                              XGBClassifier(bas...
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.01, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=3,
                                            max_leaves=None, min_child_weight=3,
                                            missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=1000, n_jobs=-1,
                                            num_parallel_tree=None,
                                            random_state=42, ...))],
                 voting='soft')

In [535]:
evaluate1(voting_clf1)

(0.9125694007160463,
 0.8648840242726705,
 array([0.084631  , 0.09662095, 0.08756588, ..., 0.36489824, 0.13132112,
        0.23953825]),
 array([0.46399044, 0.37673832, 0.15122104, ..., 0.62954438, 0.25619816,
        0.88254278]))

In [536]:
evaluate2(voting_clf2)

(0.9178366762287146,
 0.8586970455160494,
 array([0.08594899, 0.11529994, 0.09997476, ..., 0.24201136, 0.12133254,
        0.24009652]),
 array([0.30657146, 0.86818032, 0.19591877, ..., 0.81840298, 0.24446359,
        0.64437691]))

In [537]:
from sklearn.ensemble import StackingClassifier


estimators = [('model_1_lr', model_1_lr), ('model_1_rf', model_1_rf), ('model_1_xgb', model_1_xgb)]
stacking_clf1 = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(),cv=5)
stacking_clf1.fit(train_inputs,train_targets1)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


StackingClassifier(cv=5,
                   estimators=[('model_1_lr',
                                LogisticRegression(C=0.1,
                                                   class_weight='balanced',
                                                   l1_ratio=0.3, max_iter=500,
                                                   n_jobs=-1,
                                                   penalty='elasticnet',
                                                   random_state=42,
                                                   solver='saga')),
                               ('model_1_rf',
                                RandomForestClassifier(class_weight='balanced',
                                                       max_depth=10,
                                                       max_leaf_nodes=512,
                                                       max_samples=0.97,
                                                       min_impurity_decrease=0.0001,
                                                       n_estimators=1200,
                                                       n_job...
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=0.01, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=3,
                                              max_leaves=None,
                                              min_child_weight=3, missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=1000, n_jobs=-1,
                                              num_parallel_tree=None,
                                              random_state=42, ...))],
                   final_estimator=LogisticRegression())

In [538]:
from sklearn.ensemble import StackingClassifier


estimators = [('model_2_lr', model_2_lr), ('model_2_rf', model_2_rf), ('model_2_xgb', model_2_xgb)]
stacking_clf2 = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(),cv=5)
stacking_clf2.fit(train_inputs,train_targets2)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


StackingClassifier(cv=5,
                   estimators=[('model_2_lr',
                                LogisticRegression(C=0.1, l1_ratio=0.4,
                                                   max_iter=300, n_jobs=-1,
                                                   penalty='elasticnet',
                                                   random_state=42,
                                                   solver='saga')),
                               ('model_2_rf',
                                RandomForestClassifier(max_depth=12,
                                                       max_leaf_nodes=512,
                                                       max_samples=0.95,
                                                       min_impurity_decrease=0.0001,
                                                       n_estimators=1200,
                                                       n_jobs=-1,
                                                       random_state=42)),
                               ('model_2_xgb',
                                XGBClassif...
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=0.01, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=3,
                                              max_leaves=None,
                                              min_child_weight=3, missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=1000, n_jobs=-1,
                                              num_parallel_tree=None,
                                              random_state=42, ...))],
                   final_estimator=LogisticRegression())

In [539]:
evaluate1(stacking_clf1)

(0.9156512782685353,
 0.8646515056995406,
 array([0.04226174, 0.04532927, 0.04407245, ..., 0.18671274, 0.05453638,
        0.09941886]),
 array([0.29423262, 0.19664416, 0.06082517, ..., 0.53075359, 0.10654194,
        0.83279749]))

In [540]:
evaluate2(stacking_clf2)

(0.9097103367484524,
 0.8579018083929564,
 array([0.08240715, 0.09834129, 0.08605231, ..., 0.18565215, 0.09909859,
        0.1767959 ]),
 array([0.24905984, 0.88924457, 0.14700393, ..., 0.85304028, 0.19468212,
        0.68780164]))

In [541]:
test_df = pd.read_csv("/kaggle/input/x-validation/test_set_features.csv")
test_df

,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,26707,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"> $75,000",Not Married,Rent,Employed,mlyzmhmf,"MSA, Not Principle City",1.0,0.0,atmlpfrs,hfxkjkmi
1,26708,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,Non-MSA,3.0,0.0,atmlpfrs,xqwwgdyp
2,26709,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,"> $75,000",Married,Own,Employed,lrircsnp,Non-MSA,1.0,0.0,nduyfdeo,pvmttkik
3,26710,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,lrircsnp,"MSA, Not Principle City",1.0,0.0,NaN,NaN
4,26711,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,lzgpxyit,Non-MSA,0.0,1.0,fcxhlnwr,mxkfnird
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26703,53410,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,dqpwygqj,"MSA, Principle City",1.0,1.0,NaN,NaN
26704,53411,3.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,Below Poverty,Married,Rent,Employed,qufhixun,Non-MSA,1.0,3.0,fcxhlnwr,vlluhbov
26705,53412,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,qufhixun,"MSA, Not Principle City",1.0,0.0,NaN,NaN
26706,53413,3.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,bhuqouqj,"MSA, Not Principle City",1.0,0.0,NaN,NaN


In [542]:
respondent_id=test_df['respondent_id']
test_df=test_df.drop(columns=['respondent_id','hhs_geo_region','employment_industry','employment_occupation','employment_status'])
test_df['income_poverty'] = test_df['income_poverty'].replace({
    'Below Poverty':1,
    '<= $75,000, Above Poverty':2,
    '> $75,000':3
})
test_df['age_group'] = test_df['age_group'].replace({
    '18 - 34 Years':1,
    '35 - 44 Years':2,
    '45 - 54 Years':3,
    '55 - 64 Years':4,
    '65+ Years':5
})
test_df['education'] = test_df['education'].replace({
    '< 12 Years':1,
    '12 Years':2,
    'Some College':3,
    'College Graduate':4
})
# One-hot encoding
test_df = pd.get_dummies(test_df, columns=['race'], prefix='race')
test_df['sex'] = test_df['sex'].replace({
    'Female':0,
    'Male':1    
})
# One-hot encoding
test_df = pd.get_dummies(test_df, columns=['census_msa'], prefix='census_msa')
test_df['marital_status'] = test_df['marital_status'].replace({
    'Not Married':0,
    'Married':1    
})
test_df['rent_or_own'] = test_df['rent_or_own'].replace({
    'Rent':0,
    'Own':1    
})

test_df = test_df.astype(float)
import pandas as pd
from sklearn.impute import SimpleImputer

# Assuming X_train is your dataset
# X_train = pd.read_csv('your_dataset.csv') # or however you load your dataset

# Initialize the SimpleImputer with strategy as 'most_frequent' to use mode
imputer = SimpleImputer(strategy='most_frequent')

# Fit the imputer to the dataset and transform the dataset
test_df_imputed = imputer.fit_transform(test_df)

# If you want to convert the numpy array back to a DataFrame
test_df_imputed = pd.DataFrame(test_df_imputed, columns=test_df.columns)

# Display the imputed DataFrame
test_df_imputed

/tmp/ipykernel_33/1934951458.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_df['income_poverty'] = test_df['income_poverty'].replace({
/tmp/ipykernel_33/1934951458.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_df['age_group'] = test_df['age_group'].replace({
/tmp/ipykernel_33/1934951458.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, se

,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_xyz,...,rent_or_own,household_adults,household_children,race_Black,race_Hispanic,race_Other or Multiple,race_White,"census_msa_MSA, Not Principle City","census_msa_MSA, Principle City",census_msa_Non-MSA
0,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26703,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
26704,3.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
26705,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
26706,3.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [557]:
y_test1 = (voting_clf1.predict_proba(test_df_imputed))[:,1].reshape(-1)

In [558]:
y_test2 = (voting_clf2.predict_proba(test_df_imputed))[:,1].reshape(-1)

In [559]:
y_test1

array([0.22898963, 0.06359867, 0.85719285, ..., 0.26309626, 0.26789923,
       0.67374821])

In [560]:
y_test2

array([0.24538962, 0.04320382, 0.87444634, ..., 0.2642451 , 0.52292359,
       0.66559222])

In [561]:
result = pd.DataFrame({
    'respondent_id': respondent_id,
    'xyz_vaccine': y_test1,
    'seasonal_vaccine':y_test2
})

In [562]:
result

,respondent_id,xyz_vaccine,seasonal_vaccine
0,26707,0.228990,0.245390
1,26708,0.063599,0.043204
2,26709,0.857193,0.874446
3,26710,0.667989,0.825212
4,26711,0.454891,0.527824
...,...,...,...
26703,53410,0.554110,0.453293
26704,53411,0.279561,0.317064
26705,53412,0.263096,0.264245
26706,53413,0.267899,0.522924


In [563]:
result.to_csv("submission.csv", index=False) #(for making submission file)